## Load "Vector DB" and run RAG query on pre-trained instruct LLM
Simply appends retrieved chunks to query. No fancy pipeline steps or dedicated Vector DB.

In [1]:
import gc
import os
import pickle
import sys
from threading import Thread
import time
from unsloth import FastLanguageModel

import faiss
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextIteratorStreamer,
    TextStreamer,
    set_seed,
)

set_seed(1234)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Load chunk contents and embedding index
* TIL that WSL2 only has 50% of the system RAM by default. I had to increase it via `.wslconfig` so we can load both the search index and the raw chunks

In [2]:
# TODO: Database for chunks
# index = faiss.read_index("wikipedia-en.index")
index = faiss.read_index("wikipedia-en-simplifying.index")
gc.collect()

60

In [3]:
chunks = []    
embeddings_path = '/home/stefanwebb/embeddings/wikimedia/wikipedia/20231101.en'
files = [f"train-{idx:05d}-of-00041.parquet" for idx in range(41)]

print("Reading in Chunks")
for idx in range(len(files)):
    print(f"x", end="")
    chunks_file = os.path.join(embeddings_path, f'chunks-{idx:05d}-of-00041.pkl')
    with open(chunks_file, 'rb') as f:
        chunks.extend(pickle.load(f))
    gc.collect()

Reading in Chunks
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [4]:
index.ntotal, len(chunks)

(49522046, 49522046)

In [5]:
# NOTE: Following code doesn't seem to get size of all objects references by these
# sys.getsizeof(index) / 10**9, sys.getsizeof(chunks) / 10**9

### Load LLM to run queries

In [6]:
# query_model = "google/gemma-2b-it"
query_model = "/home/stefanwebb/models/llms/Mistral-7B-Instruct-v0.3"

max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/stefanwebb/models/llms/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.43.4.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0.dev20240829+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28+d444815.d20240829. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '


### Construct RAG Query and Run

In [7]:
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (

In [8]:
def run_query_streamed(query, query_model):
    system_prompt = "You are a helpful assistant who answers question truthfully to the best of your knowledge. You decline to answer if you do not know the answer."

    chat = [
        {
            "role": "system",
            "content": system_prompt,
        },
        
        {

            "role": "user",
            "content": f"{query}",
        },
    ]

    formatted_prompt = tokenizer.apply_chat_template(
        chat, tokenize=False, add_generation_prompt=True, return_tensors="pt"
    )
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    streamer = TextStreamer(
        tokenizer, skip_prompt=True, skip_special_tokens=True
    )

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=512)

In [9]:
query_encoder = SentenceTransformer("/home/stefanwebb/models/llms/multi-qa-MiniLM-L6-cos-v1")

In [10]:
import numpy as np

In [11]:
def top_k_chunks(query: str, k=1) -> str:
    """
    Find closest chunk for a given query.
    """
    embeddings = query_encoder.encode([query])
    # embeddings = embeddings / np.linalg.norm(embeddings)
    D, I = index.search(embeddings, k, params = faiss.SearchParametersIVF(nprobe=10))
    return D, I


def run_rag_query_streamed(query, query_model, k=3):
    # Retrieve most similar chunks
    D, I = top_k_chunks(query, k=k)
    # formatted_chunks = '\n\n'.join(["Document: " + chunks[i] for i in I[0]])
    formatted_chunks = ' '.join([chunks[i] for i in I[0]])
    
    # rag_query = f"Answer the query below and ground your answer in facts contained in the documents below:\n\nQuery: {query}\n\n{formatted_chunks}"

    rag_query = f"{formatted_chunks}\n\nAnswer the following question: {query}"

    # DEBUG
    # print(rag_query)
    # print("\n")

    run_query_streamed(rag_query, query_model)

    # for i, d in zip(I[0], D[0]):
    #     print(d, chunks[i])
    #     print("")

### Debug Examples
Compare answers to questions, with and without context from most similar document chunks.

In [12]:
query = "Why did Abraham Lincoln grow a beard?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model)
print()

Mistral-7B-Instruct-v0.3
Abraham Lincoln grew a beard primarily for practical reasons. In the mid-19th century, beards were common among men, but Lincoln did not have a beard during his first term as a U.S. Representative. After losing the Senate race in 1858, he grew a beard as a way to differentiate himself from his opponents in his presidential campaign in 1860. The beard also helped to hide a condition he had called "tuberculosis laryngitis," which caused his voice to be weak and hoarse. Additionally, the beard may have helped to make him appear more mature and serious, which could have been beneficial in a time of national crisis.

Mistral-7B-Instruct-v0.3 + RAG
Abraham Lincoln grew a beard in response to a written request from an 11-year-old girl named Grace Bedell. She suggested that he grow a beard to improve his appearance, and he complied with her request a few weeks before he was elected president in 1860. Lincoln met Grace Bedell in person in 1861, and he had a full beard b

In [13]:
query = "Who was Abraham Lincoln?"
D, I = top_k_chunks(query, k=100)
formatted_chunks = ' '.join([chunks[i] for i in I[0]])
print(formatted_chunks)

[CLS] abraham lincoln lincoln, abraham lincoln, abraham [SEP] [CLS] abraham lincoln lincoln, abraham lincoln, abraham [SEP] [CLS] abraham lincoln ( 1861 – 1865 ) [SEP] [CLS] abraham lincoln may also refer to : [SEP] [CLS] presidency of abraham lincoln ( 1861 - 1865 ) [SEP] [CLS] thomas lincoln ( 1778 – 1851 ) was the father of abraham lincoln. [SEP] [CLS] presidency of abraham lincoln ( 1861 – 1865 ) [SEP] [CLS] lincoln, abraham lincoln, abraham outlines [SEP] [CLS] abraham lincoln early lives of the presidents of the united states [SEP] [CLS] presidency of abraham lincoln abraham lincoln - related lists [SEP] [CLS] abraham lincoln ( 1809 – 1865 ) was the president of the united states from 1861 to 1865. [SEP] [CLS] people of power : abraham lincoln [SEP] [CLS] abraham lincoln ( ; february 12, 1809 – april 15, 1865 ) was an american lawyer, politician, and statesman who served as the 16th president of the united states from 1861 until his assassination in 1865. lincoln led the union th

In [14]:
query = "Who was Abraham Lincoln?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model, k=32)
print()

Mistral-7B-Instruct-v0.3
Abraham Lincoln was the 16th President of the United States, serving from 1861 until his assassination in 1865. He is best known for leading the nation during the American Civil War and for his role in the abolition of slavery. Lincoln was born in a log cabin in Kentucky in 1809 and grew up in Indiana and Illinois. He began his political career as a Whig, but later joined the newly formed Republican Party. Lincoln is often regarded as one of the greatest U.S. presidents and is remembered for his leadership during a time of great national crisis. He was assassinated by John Wilkes Booth at Ford's Theatre in Washington, D.C. on April 14, 1865.

Mistral-7B-Instruct-v0.3 + RAG
Abraham Lincoln was an American lawyer, politician, and statesman who served as the 16th president of the United States from 1861 to 1865. He was born on February 12, 1809, in a one-room log cabin near Hodgenville, Kentucky, to Thomas Lincoln and Nancy Hanks. Lincoln led the Union through the

In [15]:
query = "Did Abraham Lincoln have a beard?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model, k=32)
print()

Mistral-7B-Instruct-v0.3
Yes, Abraham Lincoln grew a beard during his presidency. He is often depicted with a beard in images and is known for it. Before his presidency, he did not have a beard.

Mistral-7B-Instruct-v0.3 + RAG
Yes, Abraham Lincoln grew a beard in response to a request from an 11-year-old girl named Grace Bedell, and he kept it for the rest of his presidency. The beard was a full beard, and it was first seen in public when he was elected president in 1860. Before that, he was clean-shaven.



In [16]:
query = "How are glacier caves formed?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model)
print()

Mistral-7B-Instruct-v0.3
Glacier caves, also known as moulins or ice caves, are formed in glaciers due to the melting of ice. Here's a simplified explanation of the process:

1. Meltwater: As snow accumulates on a glacier, it eventually turns into ice. Over time, this ice melts due to various factors like geothermal heat, precipitation, and air temperature. This meltwater collects in crevasses and other low spots within the glacier.

2. Enlargement of crevasses: The meltwater continues to flow, enlarging the crevasses. As the crevasse deepens, it may eventually connect with the surface of the glacier, forming a vertical shaft known as a moulin.

3. Drainage: The meltwater drains into the underlying rock or bedrock, carving out a tunnel or cave-like structure. This process continues as long as the conditions (temperature, precipitation, etc.) allow for melting and drainage.

4. Collapse: Over time, the roof of the cave may become too thin to support itself, causing it to collapse. This 

In [18]:
query = "What is a beer can pyramid?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model)
print()

Mistral-7B-Instruct-v0.3
A beer can pyramid, also known as a can pyramid or a can sculpture, is a structure made by stacking beer cans in a specific pattern to create a pyramid shape. This is often done as a fun and creative activity, and the stability of the structure depends on the number of cans used and the arrangement of the cans in the pyramid. The cans are usually empty, but some people use full cans to create a more stable structure. It's important to note that this activity should be done responsibly and safely, and the cans should be disposed of properly after the pyramid is complete.

Mistral-7B-Instruct-v0.3 + RAG
A beer can pyramid, often referred to as a beeramid, is a pyramid structure made from discarded beer cans. It is built as empty beer cans become available, and it grows gradually as the night, week, or month wears on. Typically, beeramids are temporary structures, either cleaned up or knocked over eventually.



In [19]:
query = "What is a beer can pyramid?"
D, I = top_k_chunks(query, k=10)
formatted_chunks = ' '.join([chunks[i] for i in I[0]])
print(formatted_chunks)

[CLS] a beer can pyramid, often called a beeramid as a portmanteau, is a pyramid made from discarded beer cans. beer can pyramids are built as empty beer cans became available, slowly growing as the night ( or week or month ) wears on. in most cases, though, they are temporary structures, eventually being cleaned up or accidentally knocked over. [SEP] [CLS] beeramid may refer to : beer can pyramid, a pyramid made of empty cans of beer beeramid ( comic ), a comic in the daily cardinal, a student newspaper for uw - madison [SEP] [CLS] another attempt to break the world record beer can pyramid was made with beer cans over 5 metres high and contained 10, 660 cans. it was built by the melbourne university student union in 2005, and was featured on blokesworld and in mx. [SEP] [CLS] beers pyramid once featured beers including, ales, lager, weizens, ciders, porters, and ipas. nowadays, pyramid offers six year - round beers, including hefeweizen, apricot ale, thunderhead ipa and outburst imper

In [20]:
query = "What is the current world record for a beer can pyramid?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model)
print()

Mistral-7B-Instruct-v0.3
As of my last update, the world record for the largest beer can pyramid was set by the University of Wisconsin-La Crosse Eagle's Nest Co-ed Fraternity and Sorority Life on March 23, 2019. They stacked 10,000 cans to create a pyramid that was 11.5 meters tall and 7.6 meters wide. This record was verified by Guinness World Records. However, records can change over time, so it's always a good idea to check the latest records from a reliable source.

Mistral-7B-Instruct-v0.3 + RAG
I don't have real-time data or the ability to check for updates in the world record books. However, as of the information provided, the beer can pyramid built by the Melbourne University Student Union in 2005 contained 10,660 cans, which was a significant number at that time. I don't have the current world record for a beer can pyramid. To find the most recent record, I would recommend checking the Guinness World Records or other reliable sources.



In [21]:
query = "What is the current world record for a beer can pyramid?"
D, I = top_k_chunks(query, k=3)
formatted_chunks = ' '.join([chunks[i] for i in I[0]])
print(formatted_chunks)

[CLS] another attempt to break the world record beer can pyramid was made with beer cans over 5 metres high and contained 10, 660 cans. it was built by the melbourne university student union in 2005, and was featured on blokesworld and in mx. [SEP] [CLS] a beer can pyramid, often called a beeramid as a portmanteau, is a pyramid made from discarded beer cans. beer can pyramids are built as empty beer cans became available, slowly growing as the night ( or week or month ) wears on. in most cases, though, they are temporary structures, eventually being cleaned up or accidentally knocked over. [SEP] [CLS] olympia beer was praised as one of the top 25 beers in the world in a 2012 mensjournal. com review article. [SEP]


In [22]:
query = "Which Simpsons episode featured a beer can pyramid outside of Duff Gardens?"
D, I = top_k_chunks(query, k=3)
formatted_chunks = ' '.join([chunks[i] for i in I[0]])
print(formatted_chunks)

[CLS] in the couch gag for the simpsons episode " the bob next door ", harold is shown drawing the simpson family living room during the regular title sequence. homer also asks harold to draw him a can of duff beer after he finishes with the living room. [SEP] [CLS] it was featured in season 13 the simpsons episode " weekend at burnsie's " where homer simpson ( after he smokes medicinal marijuana ) gets ready for work and pictures his world as a psychedelic wonderland. [SEP] [CLS] in popular media in the 22nd episode of season 3 of the simpsons, " the otto show, " homer simpson pulls a can of billy beer from the pocket of his " concert - going jacket, " presumably from the last concert he had attended when he was younger, and drinks the beer. [SEP]


In [23]:
query = "Who is Benjamin Geza Affleck?"
D, I = top_k_chunks(query, k=10)
formatted_chunks = ' '.join([chunks[i] for i in I[0]])
print(formatted_chunks)

[CLS] as middle name benjamin geza affleck ( born 1972 ), american actor and filmmaker [SEP] [CLS] ben geurens ( born 24 december 1979 ) is an australian actor. [SEP] [CLS] bob lechevalier : [SEP] [CLS] his younger brother faizat ghazli is also a footballer. [SEP] [CLS] he is the younger half - brother of fellow director jake nava. [SEP] [CLS] his grandson yannick cahuzac is a professional footballer. [SEP] [CLS] ben zabo ( born arouna moussa coulibaly, january 24, 1979 ) is a malian afrobeat - musician from bamako. he was born at tominian in the segou region. he released his self - titled debut album on the international record company glitterhouse records in 2012. [SEP] [CLS] he is the brother of renato marazzi. [SEP] [CLS] in 1988 meretzky had his son daniel meretzky. [SEP] [CLS] he is the son of late broadcaster bob arbogast. [SEP]


In [24]:
query = "Who is Benjamin Geza Affleck?"

print("Mistral-7B-Instruct-v0.3")
run_query_streamed(query, model)
print()

print("Mistral-7B-Instruct-v0.3 + RAG")
run_rag_query_streamed(query, model)
print()

Mistral-7B-Instruct-v0.3
Benjamin Geza Affleck is a well-known American actor, filmmaker, and screenwriter. He was born on August 15, 1972. Affleck gained fame for his roles in films such as "Good Will Hunting," for which he won an Academy Award for Best Original Screenplay along with Matt Damon, and "Argo," for which he won the Academy Award for Best Picture as a producer. He is also known for his roles in films like "Dazed and Confused," "Chasing Amy," "Gone Baby Gone," and "The Town." In addition to acting, Affleck has directed films such as "Gone Baby Gone," "The Town," and "Argo." He is also a political activist and has been involved in various philanthropic efforts.

Mistral-7B-Instruct-v0.3 + RAG
Benjamin Geza Affleck is an American actor and filmmaker, born in 1972. He is known for his work in films such as "Good Will Hunting," "Argo," and "Gone Baby Gone." He has won multiple awards, including two Academy Awards, for his acting and directing work.



### Investigating whether failure to retrieve relevant chunks is due to search index or document/query encoder

In [25]:
query = "What is the current world record for a beer can pyramid?"
D, I = top_k_chunks(query, k=10)
formatted_chunks = ' '.join([chunks[i] for i in I[0]])
print(formatted_chunks)


[CLS] another attempt to break the world record beer can pyramid was made with beer cans over 5 metres high and contained 10, 660 cans. it was built by the melbourne university student union in 2005, and was featured on blokesworld and in mx. [SEP] [CLS] a beer can pyramid, often called a beeramid as a portmanteau, is a pyramid made from discarded beer cans. beer can pyramids are built as empty beer cans became available, slowly growing as the night ( or week or month ) wears on. in most cases, though, they are temporary structures, eventually being cleaned up or accidentally knocked over. [SEP] [CLS] olympia beer was praised as one of the top 25 beers in the world in a 2012 mensjournal. com review article. [SEP] [CLS] beers pyramid once featured beers including, ales, lager, weizens, ciders, porters, and ipas. nowadays, pyramid offers six year - round beers, including hefeweizen, apricot ale, thunderhead ipa and outburst imperial ipa. pyramid also offers seasonal beers, including curv

In [26]:
I[0]

array([44419780, 44419778, 46332100, 42621542, 32287273, 48056004,
       10667840, 44684914, 15182634,  6031383])

In [27]:
chunks[44419780 - 1]

'[CLS] on 23 september 2000, the malaysian can team, consisting of 12 college students from the inti college subang jaya, malaysia built a free standing can pyramid created from 9, 455 empty aluminium drink cans in 24 minutes at the mid valley megamall in kuala lumpur, malaysia. it had a square base of cans, measuring. this feat made a successful entry into the guinness world record and to - date this record has yet to be broken. [SEP]'

In [28]:
import numpy as np

In [29]:
# Comparing similarity of query to retrieved chunks and the non-retrieved chunk that contains the necessary fact
emb_query = query_encoder.encode([query])
emb_chunk0 = query_encoder.encode([chunks[44419780]])
emb_chunk1 = query_encoder.encode([chunks[44419780 - 1]])
emb_chunk2 = query_encoder.encode([chunks[15182634]])
emb_chunk3 = query_encoder.encode([chunks[32287273]])

In [30]:
for emb in [emb_chunk0, emb_chunk1, emb_chunk2, emb_chunk3]:
    score1 = np.dot(emb_query.flatten(), emb.flatten())
    print(score1) 

0.80006427
0.6656804
0.5796839
0.61451036


In [24]:
query = "What is the current world record for a beer can pyramid?"
D, I = top_k_chunks(query, k=1000)
I[0]

array([44419780, 44684914, 16419147, 20744553,  6416773, 19033151,
       26014709, 41376691, 34361474, 26496274, 42685118,  5640383,
       44085428,  5263339,   973686, 47448553, 23267869, 46842458,
       34141572, 37570376,  8779913,  8815147, 25444862, 31087900,
       16081675, 19195266,  4185976, 34936540,  4689674, 16081669,
       36622096, 30384421, 15366051,  5588299, 47866189, 27002093,
         340457, 18100694, 32287274, 16084566,   124748, 34141574,
       47848535, 19965494, 44085405, 21991699, 35899911, 49433051,
       16767536, 36497538, 45207851,  4801097,   124775, 36835559,
        3084689, 45157266, 17168589, 29561178, 12708203,   635835,
        7181068, 16212102,   124766, 12174024, 45156942, 45157245,
       25905764, 41505238, 28724613, 34936532, 45157269, 42145038,
       27735876,    54510, 23466812, 45100479, 44948833, 47380163,
       35153054, 32182806,  2360932, 45196960, 14212395, 15515414,
        1664053, 42820844, 41376750, 18919372, 46878533, 36575

In [31]:
# Even fetching the top 1000 matches in index, required chunk isn't found... :(
44419780 in I[0], 44419779 in I[0], 44419778 in I[0]

(True, False)

In [26]:
query = "What is the current world record for a beer can pyramid?"
D, I = top_k_chunks(query, k=1000)

scores = []
emb_query = query_encoder.encode([query])
for idx in I[0]:
    emb= query_encoder.encode([chunks[idx]])
    score = np.dot(emb_query.flatten(), emb.flatten())
    scores.append(score)

print(list(reversed(list(sorted(scores)))))

[0.80006427, 0.6221438, 0.56738615, 0.56650317, 0.5571245, 0.55555916, 0.55080813, 0.5476148, 0.54552424, 0.5436713, 0.53899646, 0.5366755, 0.5345394, 0.53018284, 0.52894855, 0.52799815, 0.5221437, 0.52118266, 0.52045053, 0.5203744, 0.51641655, 0.5130265, 0.5127297, 0.5116299, 0.50966203, 0.50895905, 0.50880957, 0.5081908, 0.50676394, 0.5065583, 0.50640064, 0.50514567, 0.5045469, 0.5041433, 0.50370705, 0.5032388, 0.5027695, 0.5012673, 0.5010059, 0.50094634, 0.4988805, 0.49821705, 0.4972093, 0.4957267, 0.49480417, 0.49455714, 0.4907418, 0.4900855, 0.48919064, 0.4891385, 0.48910388, 0.4890645, 0.48905128, 0.4884631, 0.48836598, 0.4882414, 0.48771375, 0.4874601, 0.48642403, 0.48611897, 0.48571566, 0.4854838, 0.4842167, 0.48386508, 0.48254412, 0.48151135, 0.48133773, 0.48129317, 0.48040962, 0.47928306, 0.47695112, 0.4769107, 0.47681057, 0.47648796, 0.47629863, 0.47509062, 0.47474012, 0.47249687, 0.47248125, 0.4720722, 0.4720341, 0.47153708, 0.46922535, 0.4686184, 0.46785474, 0.46764272, 0.